In [1]:
import pandas as pd

# Preprocessing

In [2]:
steam_df = pd.read_csv('steam-200k.csv', encoding='utf-8')

player_df = pd.DataFrame(steam_df['user_id'].unique(), columns=['user_id'])
player_df['id'] = player_df.index + 1
player_df = player_df.reindex(columns=['id', 'user_id'])

game_df = pd.DataFrame(steam_df['game_title'].unique(), columns=['game_title'])
game_df['id'] = game_df.index + 1
game_df = game_df.reindex(columns=['id', 'game_title'])

play = {'from_id': steam_df['user_id'], 'to_id': steam_df['game_title'], 'play_hour': steam_df['play_hour']}
play_df = pd.DataFrame(data=play)
uid_dic = dict(zip(player_df['user_id'], player_df['id']))
game_dic = dict(zip(game_df['game_title'], game_df['id']))
play_df['from_id'] = play_df['from_id'].replace(uid_dic)
play_df['to_id'] = play_df['to_id'].replace(game_dic)

In [3]:
play_df.to_csv('PLAY.csv', encoding='utf-8', index=False)
player_df.to_csv('Player.csv', encoding='utf-8', index=False)
game_df.to_csv('Game.csv', encoding='utf-8', index=False)

# Connect to database and get data

In [4]:
from neo4j import GraphDatabase

In [5]:
# Information of connection
URI = 'neo4j+s://eb65d4b0.databases.neo4j.io'
USER = 'neo4j'
PASSWORD = 'Euk7C4-PnXiMy0ZFmBqG_pZGzshLkQ9NLb0P6ZKHVHQ'

In [6]:
# Make connection
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
session = driver.session()

# Q1

In [7]:
query = 'match (u:Player)-[p:PLAY]->(g:Game {game_title:"Sid Meier\'s Civilization V"}) return u,p,g'
records = session.run(query)

# Extract result data from neo4j
play_hour = []
player = []
for record in records:
    play_hour.append(record['p']['play_hour'])
    player.append(record['u']['user_id'])

# Close connection
driver.close()

## Q1-1

In [8]:
print('總共有', len(player), '人玩過 Sid Meier\'s Civilization V')

總共有 554 人玩過 Sid Meier's Civilization V


## Q1-2

In [9]:
import numpy as np
print('平均遊玩時間為', round(np.mean(play_hour), 2), '小時')

平均遊玩時間為 180.18 小時


# Q2

In [10]:
import json

In [11]:
query = '''
match (p:Player)-[r:PLAY]->(g:Game) 
with g.game_title AS game_name, avg(r.play_hour) as average_play_hour 
return game_name, average_play_hour'''
records = session.run(query)

# Extract result data from neo4j
game_name = []
avg_play_hour = []
for record in records:
    game_name.append(record['game_name'])
    avg_play_hour.append(record['average_play_hour'])

# Get games whether game_play_hour is greater than 20 hours
game_play_hour = dict(zip(game_name, avg_play_hour))
game20 = {key: value for key, value in game_play_hour.items() if value>20}

with open('game_play_hour>20.json', 'w') as f:
    json.dump(game20, f)

# Close connection
driver.close()